In [1]:
# Install Gradio
!pip install gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.4 MB/s eta 0:00:00


**Libraries**

In [2]:
from PIL import Image, ImageTk
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import os
import gradio as gr

**Mounting Drive & Defining Paths**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Load The Model**

In [4]:
# List of model paths
model_paths = [
    "/content/drive/MyDrive/LungCancer_CNN_model_v2.h5",
    "/content/drive/MyDrive/LungCancer_VGG16_model_v2.h5",
    "/content/drive/MyDrive/LungCancer_InceptionV3_model_v2.h5",
    "/content/drive/MyDrive/LungCancer_ResNet50_model_v2.h5",
    "/content/drive/MyDrive/LungCancer_CNN_model_v3.h5"
]

# Dictionary to store model paths and their test accuracies
models_info = {
    'CNN Model V1': {'path': model_paths[0]},
    'VGG16 Model (Best Performer)': {'path': model_paths[1]},
    'InceptionV3 Model': {'path': model_paths[2]},
    'ResNet50 Model': {'path': model_paths[3]},
    'CNN Model V2': {'path': model_paths[4]}
}

# Class labels (update based on your dataset)
class_labels = {0: "Adenocarcinoma", 1: "Benign", 2: "Squamous Cell Carcinoma"}

# Global variable to store the loaded model
loaded_model = None

def choose_and_load_model(model_name):
    global loaded_model
    model_path = models_info[model_name]['path']

    # Check if the model file exists
    if not os.path.exists(model_path):
        return f"Model file not found at {model_path}"

    # Load the selected model
    loaded_model = load_model(model_path)
    return f"Loaded {model_name} from {model_path}"

**Make The GUI**

In [5]:
def predict_lung_cancer(img):
    global loaded_model
    if loaded_model is None:
        return "No model loaded. Please select a model first."

    # Preprocess the image
    img = img.resize((150, 150))  # Resize image to 150x150
    img_array = np.array(img) / 255.0  # Rescale pixel values
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Predict using the model
    predictions = loaded_model.predict(img_array)
    predicted_class_index = np.argmax(predictions)
    confidence = np.max(predictions)

    # Get class label
    predicted_class = class_labels.get(predicted_class_index, "Unknown")
    return f"Prediction: {predicted_class} | Confidence: {confidence:.2f}"

In [6]:
# Create a Gradio interface
def gradio_interface():
    model_names = list(models_info.keys())
    dropdown = gr.Dropdown(choices=model_names, label="Select Model")
    image_input = gr.Image(type="pil", label="Upload Image")
    output = gr.Textbox(label="Output")

    def combined_function(model_name, img):
        load_message = choose_and_load_model(model_name)
        prediction_message = predict_lung_cancer(img)
        return f"{load_message}\n{prediction_message}"

    gr.Interface(
        fn=combined_function,
        inputs=[dropdown, image_input],
        outputs=output
    ).launch()

**Launch The App**

In [7]:
# Launch the interface
gradio_interface()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f798cc2b57f0e10430.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
